# 04 - Test and Deploy Training Pipeline to Vertex Pipelines

The purpose of this notebook is to compile the TFX pipeline and run the pipeline on `Vertex Pipelines`. The notebook covers the following tasks:
1. Run unit tests.
2. Test the pipeline locally using a local runner.
3. Set the pipeline deployment configuration.
4. Build a container image.
6. Compile the TFX pipeline.
5. Submit the pipeline job to `Vertex Pipelines`.

## Installation

Install the latest version of Vertex SDK.

In [ ]:
import sys
import os


# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = '--user'
else:
    USER_FLAG = ''

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG

Install the latest GA version of *google-cloud-storage* library as well.

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

Install deep learning dependencies

In [ ]:
! pip3 install -U tfx==0.30.0 $USER_FLAG
! pip3 install -r requirements.txt $USER_FLAG

Install pytest

In [ ]:
! pip3 install pytest $USER_FLAG

### Restart the kernel

Once you've installed the Vertex SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import kfp
import tfx

print("Tensorflow Version:", tfx.__version__)
print("KFP Version:", kfp.__version__)

### Setup your Google Cloud project

Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

In [ ]:
PROJECT_ID = "[your-project-id]"  #@param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)
    
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex. Not all regions provide support for all Vertex services. For the latest support per region, see the [Vertex locations documentation](https://cloud.google.com/ai-platform-unified/docs/general/locations)

In [ ]:
REGION = 'us-central1'  #@param {type: "string"}

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Setup a bucket for the experiment

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  #@param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

In [ ]:
!rm -r src/raw_schema/.ipynb_checkpoints/

In [ ]:
BQ_LOCATION = 'US'
BQ_DATASET_NAME = 'playground_us' # Change to your BQ datasent name.
BQ_TABLE_NAME = 'chicago_taxitrips_prep'

VERSION = 'v01'
DATASET_DISPLAY_NAME = 'chicago_taxi_tips'
MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}_classifier_{VERSION}'

CICD_IMAGE_NAME = 'cicd:latest'
CICD_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{CICD_IMAGE_NAME}"

## Build CI/CD  Container Image for Cloud Build

This is the runtime environment where the steps of testing and deploying the model will be executed.

In [ ]:
! echo $CICD_IMAGE_URI

In [ ]:
! gcloud builds submit --tag $CICD_IMAGE_URI build/. --timeout=15m --machine-type=e2-highcpu-8

## 1. Run the Pipeline CICD steps locally

### Set pipeline configurations for the local run

In [ ]:
os.environ["DATASET_DISPLAY_NAME"] = DATASET_DISPLAY_NAME
os.environ["MODEL_DISPLAY_NAME"] =  MODEL_DISPLAY_NAME
os.environ["PROJECT"] = PROJECT_ID
os.environ["REGION"] = REGION
os.environ["BQ_LOCATION"] = BQ_LOCATION
os.environ["BQ_DATASET_NAME"] = BQ_DATASET_NAME
os.environ["BQ_TABLE_NAME"] = BQ_TABLE_NAME
os.environ["GCS_LOCATION"] = f"{BUCKET_NAME}/vertex_demo/e2e_tests"
os.environ["TRAIN_LIMIT"] = "1000"
os.environ["TEST_LIMIT"] = "100"
os.environ["UPLOAD_MODEL"] = "0"
os.environ["ACCURACY_THRESHOLD"] = "0.1"
os.environ["BEAM_RUNNER"] = "DirectRunner"
os.environ["TRAINING_RUNNER"] = "local"

In [ ]:
from src.tfx_pipelines import config
for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

### Run unit tests

In [ ]:
! py.test src/tests/datasource_utils_tests.py -s

In [ ]:
! py.test src/tests/model_tests.py -s

### Run e2e pipeline test

In [ ]:
! py.test src/tests/pipeline_deployment_tests.py::test_e2e_pipeline -s

### Set the pipeline configurations for the Vertex AI run

In [ ]:
os.environ["DATASET_DISPLAY_NAME"] = DATASET_DISPLAY_NAME
os.environ["MODEL_DISPLAY_NAME"] = MODEL_DISPLAY_NAME
os.environ["PROJECT"] = PROJECT_ID
os.environ["REGION"] = REGION
os.environ["GCS_LOCATION"] = f"{BUCKET_NAME}/vertex_demo"
os.environ["TRAIN_LIMIT"] = "85000"
os.environ["TEST_LIMIT"] = "15000"
os.environ["BEAM_RUNNER"] = "DataflowRunner"
os.environ["TRAINING_RUNNER"] = "vertex"
os.environ["TFX_IMAGE_URI"] = f"gcr.io/{PROJECT_ID}/{DATASET_DISPLAY_NAME}:{VERSION}"

In [ ]:
from src.tfx_pipelines import config

import importlib
importlib.reload(config)

for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

### Build container image

This is the tfx runtime environment for the training pipeline steps.

In [ ]:
! echo $TFX_IMAGE_URI

In [ ]:
! gcloud builds submit --tag $TFX_IMAGE_URI . --timeout=15m --machine-type=e2-highcpu-8

### Compile pipeline

In [ ]:
from src.tfx_pipelines import runner

pipeline_definition_file = f'{config.PIPELINE_NAME}.json'
pipeline_definition = runner.compile_training_pipeline(pipeline_definition_file)
pipeline_definition

In [ ]:
PIPELINES_STORE = f"{BUCKET_NAME}/vertex_demo/compiled_pipelines"
! gsutil cp {pipeline_definition_file} {PIPELINES_STORE}

### Submit run to Vertex AI Pipelines

In [ ]:
from kfp.v2.google.client import AIPlatformClient

pipeline_client = AIPlatformClient(
    project_id=PROJECT_ID, region=REGION)
                 
pipeline_client.create_run_from_job_spec(
    job_spec_path=pipeline_definition_file,
    parameter_values={
        'learning_rate': 0.003,
        'batch_size': 512,
        'hidden_units': '128,128',
        'num_epochs': 30,
    }
)

## 2. Execute the Pipeline Deployment CI/CD routine in Cloud Build

In [ ]:
REPO_URL = "https://github.com/ksalama/ucaip-labs.git" # Change to your github repo.
BRANCH = "main"

GCS_LOCATION = f"{BUCKET_NAME}/vertex_demo/"
TEST_GCS_LOCATION = f"{BUCKET_NAME}/vertex_demo/e2e_tests"
TRAIN_LIMIT = 1000
TEST_LIMIT = 100
UPLOAD_MODEL = 0
ACCURACY_THRESHOLD = 0.1
BEAM_RUNNER = "DataflowRunner"
TRAINING_RUNNER = "vertex"
VERSION = 'tfx-0-30'
PIPELINE_NAME = f'{MODEL_DISPLAY_NAME}_train_pipeline'
PIPELINES_STORE = os.path.join(GCS_LOCATION, "compiled_pipelines")

TFX_IMAGE_URI = f"gcr.io/{PROJECT_ID}/{DATASET_DISPLAY_NAME}:{VERSION}"

SUBSTITUTIONS=f"""\
_REPO_URL='{REPO_URL}',\
_BRANCH={BRANCH},\
_CICD_IMAGE_URI={CICD_IMAGE_URI},\
_PROJECT={PROJECT_ID},\
_REGION={REGION},\
_GCS_LOCATION={GCS_LOCATION},\
_TEST_GCS_LOCATION={TEST_GCS_LOCATION},\
_BQ_LOCATION={BQ_LOCATION},\
_BQ_DATASET_NAME={BQ_DATASET_NAME},\
_BQ_TABLE_NAME={BQ_TABLE_NAME},\
_DATASET_DISPLAY_NAME={DATASET_DISPLAY_NAME},\
_MODEL_DISPLAY_NAME={MODEL_DISPLAY_NAME},\
_TRAIN_LIMIT={TRAIN_LIMIT},\
_TEST_LIMIT={TEST_LIMIT},\
_UPLOAD_MODEL={UPLOAD_MODEL},\
_ACCURACY_THRESHOLD={ACCURACY_THRESHOLD},\
_BEAM_RUNNER={BEAM_RUNNER},\
_TRAINING_RUNNER={TRAINING_RUNNER},\
_TFX_IMAGE_URI={TFX_IMAGE_URI},\
_PIPELINE_NAME={PIPELINE_NAME},\
_PIPELINES_STORE={PIPELINES_STORE}\
"""

!echo $SUBSTITUTIONS

In [ ]:
!gcloud builds submit --no-source --timeout=60m --config build/pipeline-deployment.yaml --substitutions {SUBSTITUTIONS} --machine-type=e2-highcpu-8